In [1]:
import matplotlib
matplotlib.rcParams['text.usetex'] = True
matplotlib.rcParams['font.family'] = 'serf'
import matplotlib.pyplot as plt
import numpy as np

In [2]:
def solve_ftcs(f, v, dx, dt):
    f_next = np.zeros(len(f))
    f_next = f - v * dt / 2.0 / dx * (np.roll(f,-1) - np.roll(f, 1))
    return f_next

def solve_lax(f, v, dx, dt):
    f_next = np.zeros(len(f))
    f_next = 0.5 * (np.roll(f, 1) + np.roll(f, -1)) - v * dt / 2.0 / dx * (np.roll(f,-1) - np.roll(f, 1))
    return f_next

def solve_upwind(f, v, dx, dt):
    # f_next = np.zeros(len(f))
    f_next = f - v * dt / dx * (f - np.roll(f, 1))
    return f_next

def solve_leapfrog(f, f1, v, dx, dt):
    f_next = np.zeros(len(f))
    f_next = f1 - v * dt / dx * (np.roll(f,-1) - np.roll(f, 1))
    return f_next

def solve_lax_wendroff(f, v, dx, dt):
    f1 = 0.5 * (np.roll(f, 1) + f) - v * dt / 2.0 / dx * (f - np.roll(f, 1))
    f2 = 0.5 * (f + np.roll(f, -1)) - v * dt / 2.0 / dx * (np.roll(f, -1) - f)
    f_next = f - v * dt / dx * (f2 - f1)
    return f_next

In [2]:
def solve_lax_2d(f, vx, vy, dx, dy, dt):
    f_next = 0.25 * (np.roll(f, 1, axis=0) + np.roll(f, -1, axis=0) +
                     np.roll(f, 1, axis=1) + np.roll(f, -1, axis=1)) - \
              vx * dt / 2.0 / dx * (np.roll(f,-1, axis=1) - np.roll(f, 1, axis=1)) - \
              vy * dt / 2.0 / dy * (np.roll(f,-1, axis=0) - np.roll(f, 1, axis=0))
    return f_next

def upwind_2d(f, vx, vy, dx, dy, dt):
    f_next = f - vx * dt / dx * (f - np.roll(f, 1, axis=0)) - vy * dt / dy * (f - np.roll(f, 1, axis=1))
    return f_next

def solve_lax_wendroff_2d(f, vx, vy, dx, dy, dt):
    f1 = 0.5 * (np.roll(f, 1, axis=0) + f) - vx * dt / 2.0 / dx * (f - np.roll(f, 1, axis=0))
    f2 = 0.5 * (f + np.roll(f, -1, axis=0)) - vx * dt / 2.0 / dx * (np.roll(f, -1, axis=0) - f)
    f3 = 0.5 * (np.roll(f, 1, axis=1) + f) - vy * dt / 2.0 / dy * (f - np.roll(f, 1, axis=1))
    f4 = 0.5 * (f + np.roll(f, -1, axis=1)) - vy * dt / 2.0 / dy * (np.roll(f, -1, axis=1) - f)
    f_next = f - vx * dt / dx * (f2 - f1) - vy * dt / dy * (f4 - f3)
    return f_next

def solve_fcts_diffusion(f, D, dx, dt):
    f_next = np.zeros(len(f))
    f_next = f + D * dt / dx**2 * (np.roll(f,-1) - 2 * f + np.roll(f, 1))
    return f_next

In [3]:
print(np.roll(y, -1, axis=0))

NameError: name 'y' is not defined

In [24]:
nx = 1000
lx = 1.0
xs = np.linspace(0, lx, nx)
dx = xs[1] - xs[0]
D = 1.0
# dt = 0.5 * dx**2 / D
dt = 1.0e-4

f = np.exp(-((xs - 0.5) / 0.1)**2)
# f = f1

from scipy import sparse
from scipy.sparse.linalg import spsolve
alpha = D * dt / dx**2
A = sparse.spdiags(np.array([-alpha * np.ones(nx), 1 + 2 * alpha * np.ones(nx), -alpha * np.ones(nx)]),
                   np.array([-1, 0, 1]), nx, nx)

def latex_float(f):
    float_str = "{0:.2g}".format(f)
    if "e" in float_str:
        base, exponent = float_str.split("e")
        return r"{0} \times 10^{{{1}}}".format(base, int(exponent))
    else:
        return float_str

freq = 1
for i in range(100):
    t = i*dt
    if i % freq == 0:
        plt.plot(xs, f)
        ax = plt.gca()
        ax.set_xlabel("$x$")
        ax.set_ylabel("$u$")
        ax.set_ylim(-2.0, 3.0)
        ax.set_xlim(0.0, 1.0)
        ax.set_title(f"$t = {latex_float(t)}$")
        ax.axhline(0.0, color='r', ls='--')
        plt.savefig("fig%03d.png" % (i/freq), bbox_inches='tight')
        plt.close()
    # f = solve_fcts_diffusion(f, D, dx, dt)
    f = spsolve(A, f)

/usr/lib/python3.11/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:229: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  warn('spsolve requires A be CSC or CSR matrix format',
/usr/lib/python3.11/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:229: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  warn('spsolve requires A be CSC or CSR matrix format',
/usr/lib/python3.11/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:229: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  warn('spsolve requires A be CSC or CSR matrix format',
/usr/lib/python3.11/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:229: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  warn('spsolve requires A be CSC or CSR matrix format',
/usr/lib/python3.11/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:229: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  warn('spsolve r